### Segmenting and clustering neighbourhoods for Toronto

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download all dependencies we need</a>

2. <a href="#item2">Build scraping code</a>

3. <a href="#item3">Create dataframe</a>

4. <a href="#item4">Submit a link of notebook on Github repository</a>

</font>
</div>

Downloading all the dependencies that we will need

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
print('Downloaded!')

Downloaded!


Install BeautifulSoup4

In [2]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 1.0MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.7MB 1.0MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [45]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import os

def make_soup(url):
    thepage=urllib.request.urlopen(url)
    soupdata=BeautifulSoup(thepage,'html.parser')
    return soupdata

tabledatasaved=''
soup=make_soup('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
for record in soup.findAll('tr'):
    tabledata=''
    for data in record.findAll('td'):
        tabledata=tabledata+','+data.text
    if len(tabledata)!=0:
        tabledatasaved=tabledatasaved+'\n'+tabledata[1:]
        
header='Postal_code,Borough,Neighborhood'
file=open(os.path.expanduser('Toronto.csv'),'wb')
file.write(bytes(header,encoding='ascii',errors='ignore'))
file.write(bytes(tabledatasaved,encoding='ascii',errors='ignore'))

toronto_df=pd.read_csv('Toronto.csv')

toronto_df.head()


,Postal_code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [46]:
# Get names of indexes for which column Borough has value Not assigned
toronto_df['Borough'] = toronto_df['Borough'].astype('str')
toronto_filtered_df = toronto_df[toronto_df['Borough']== 'Not assigned' ].index
# Delete these row indexes from dataFrame
toronto_df.drop(toronto_filtered_df , inplace=True)
toronto_df.head()

,Postal_code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [105]:
toronto1_df=toronto_df.groupby('Postal_code', as_index=False,sort=False).agg(lambda x: ', '.join(set(x.astype(str))))
toronto1_df.head()

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Not assigned


In [103]:
cond = toronto1_df.Neighborhood == 'Not assigned'

toronto1_df.Neighborhood[cond] = toronto1_df.Borough[cond]
toronto1_df.head()

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [104]:
toronto1_df.shape

(133, 3)